# General Setup

In [1]:
##### Following lines need to be written in the terminal for permisson of live access to RhymeZone #####
##### also in setup.txt for copy pasting #####

##### CREATING A NEW CONDA ENV ADVISED #####
# cd Echo_Canyon
# conda create --name Echo_Canyon 
# conda activate Echo_Canyon
# pip install -r requirements.txt
# after you are done: conda env remove --name Echo_Canyon

##### YOU MAY HAVE TO RUN THESE IN THE TERMINAL #####
# sudo apt-get update
# sudo apt install chromium-chromedriver
# conda install -c conda-forge python-chromedriver-binary

##### THEN JUST CLICK RUN ALL #####


# Saving
import os
from os.path import exists
import shutil

# Paragraph Generator
#!pip install transformers  
#!pip install tf-models-official 

from transformers import AutoConfig, AutoTokenizer, TFAutoModelForCausalLM, default_data_collator, pipeline
os.environ["CUDA_VISIBLE_DEVICES"]="-1" #to deactive GPU
model_name = "gpt2" # best model after trying 'distilgpt2' for general text generation
import transformers # importing the huggingface transformers library
from transformers import pipeline

# Line Generator
###

# Word Replacer

mask_filler = pipeline("fill-mask")

# Rhyming with "pronunciation"
#!pip install pronouncing
import pronouncing



/opt/anaconda3/envs/echo_canyon/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilroberta-base and revision ec58a5b (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the 

In [2]:
curr_path = os.getcwd()  #assuming the jupyter notebook is started from inside the Echo_Canyon, so Echo_Canyon is root

In [3]:
curr_path

'/Users/tatlikazan-rv/Documents/thesis_documents/relevant-projects/Echo_Canyon'

In [4]:
# Rhyming with "RhymeZone.com"
#!pip install selenium
#!pip install bs4

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)
#driver = webdriver.Chrome('chromedriver',options=chrome_options) older version
#driver = webdriver.Chrome(curr_path + '/chromedriver') 

from bs4 import BeautifulSoup 
from urllib.parse import quote_plus
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
#from webdriver_manager.chrome import ChromeDriverManager

# Saving Lyrics

In [5]:
def save_lyrics(lyrics):   
    i = 0
    path = curr_path + '/gen_lyrics/lyrics_current.txt'

    while exists(path):
        i += 1
        path = curr_path + '/gen_lyrics/lyrics_' + str(i) + '.txt'  
        if not exists(path):
            original = curr_path + '/gen_lyrics/lyrics_current.txt'
            target = path
            shutil.copyfile(original, target)
            break

    with open(curr_path + '/gen_lyrics/lyrics_current.txt', 'w') as f:
        f.write(lyrics) 

    print("\n")
    #print(lyrics)
    return lyrics

# Loading Lyrics

In [6]:
def load_gen_lyrics():
    with open(curr_path + '/gen_lyrics/lyrics_current.txt', 'r') as f:
        lyrics = f.read()#.replace("\n", " ")
        f.close()
    lines = lyrics.split("\n")
    lyrics = "  ".join(lines)
    enum_print_lines(lines)

    print("\n")
    return lyrics

# Paragraph Generator

In [7]:
# GENERATING

def get_input():
    genre = int(input("Choose a genre from the following:\n\
0: blues\n1: country\n2: jazz\n3: metal\n4: pop\n5: rock\n6: my chosen artists (Blues Brothers)\nAnswer by number: "))
    prompt = input("Write down a line for your song OR just push enter to skip this step (dont choose this if you are starting from scracth): ")
    print("\n")
    return genre, prompt

def init_genre_model(genre):
    print("Setting up genre specific model...")
    model_path = curr_path + "/saved_model/"
    genre_names = ["blues", "country", "jazz", "metal", "pop", "rock", "my_artists"]  # count from 0
    model_path += genre_names[genre]
    #model = TFAutoModelForCausalLM.from_pretrained(model_path)
    model = TFAutoModelForCausalLM.from_pretrained("brunokreiner/lyrics-bert", from_pt=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generator = pipeline('text-generation',model=model, tokenizer=tokenizer)
    print("Model Path: " + model_path)
    print("Done!")
    print("\n")

    return generator, tokenizer

def generate_lyrics(prompt, generator):

    # Too random with 1-2 words, doesnt get the context
    # Performs better with one sentence
    # Too restrictive and causes repetititions with a paragraph

    print("Generating...")
    n_generations = 3 # runs 
    
    if len(prompt) < 3:
        prompt = int(prompt)
        lyrics = lines[prompt]
    else:
        lyrics = prompt
     
    for i in range(n_generations):
        
        lyrics += generator(lyrics[-100:], return_full_text = False)[0]["generated_text"]
    
    lines = lyrics.split("\n")
    tmp = []
    for line in lines:
        if line != '':
            tmp.append(line)

    lyrics = '\n'.join(tmp)    
    return lyrics

def enum_print_lyrics(lyrics):
    print("\n")
    print("Your lyrics:")
    lines = lyrics.split("\n")
    for i in range(len(lines)):
        print(i, ": ", lines[i])
    print("\n")

def erase_lines(lyrics):

    lines = lyrics.split("\n")
    lyrics = '\n'.join(lines)

    enum_print_lyrics(lyrics)

    retry = input("Write r to regenarate OR just push enter to keep going: ")
    if retry == ['']:
        pass
    elif retry == "r":
        print("RESET")
        print("\n")
        lets_write_a_song()      
        return lyrics

    placeholder = input("Give the number of lines you want to add a place holder instead (e.g.: '1,2,4,8') OR just push enter to skip: ").split(",")
    if placeholder != ['']:
        placeholder = [int(i) for i in placeholder]
    
    tmp = []
    for i in range(len(lines)):
        if i in placeholder:
            tmp.append("<placeholder>")
        else:
            tmp.append(lines[i])
    
    lines = tmp
    
    erase = input("Give the number of lines you want to erase (e.g.: '1,2,4,8') OR just push enter to skip: ").split(",")
    if erase != ['']:
        erase = [int(i) for i in erase]
    
    tmp = []
    for i in range(len(lines)):
        if i not in erase:
            tmp.append(lines[i])
    
    lines = tmp
    lyrics = '\n'.join(lines) 

    enum_print_lyrics(lyrics)
    
    return lyrics

def longer_song():
    print("\n")
    longer = int(input("Do you want to make your song longer?\n0: no\n1: yes\nAnswer (with number): "))
    print("\n")
    return longer


def choose_line_as_prompt(lyrics):
    lines = lyrics.split("\n")
    print("\n")
    choice = input("Give the number of the line you want to base new lines on OR write down another line: ")
    print("\n")
    if len(choice) < 3:
        choice = int(choice)
        prompt = lines[choice]
    else:
        prompt = choice
    return prompt

def generate_lyrics_demo(lyrics):
    genre, prompt = get_input()
    generator, tokenizer = init_genre_model(genre)
    
    if prompt != "":
        if lyrics == "":
            lyrics = generate_lyrics(prompt, generator)
        if lyrics != "":
            lyrics += "\n" + generate_lyrics(prompt, generator)
    lyrics = erase_lines(lyrics)
    longer = longer_song()

    while longer == 1:
        prompt = choose_line_as_prompt(lyrics)
        lyrics = lyrics + "\n" + generate_lyrics(prompt, generator)
        lyrics = erase_lines(lyrics)    
        longer = longer_song()

    lyrics = save_lyrics(lyrics)

    return lyrics

# Line Generator

In [8]:
pass # with Seq2seq, unfotunately didnt have the time

# Swapping Lines

In [9]:
def swap_lines(lyrics):
    print("\n") 
    lines = lyrics.split("\n")
    enum_print_lines(lines)
    print("\n")

    coord = input("Give the number of the lines you want to swap (e.g. 1,2): ").split(",")
    coord = [int(i) for i in coord]
    
    #print(coord)

    tmp = lines[coord[0]]
    lines[coord[0]] = lines[coord[1]]
    lines[coord[1]] = tmp

    lyrics = "\n".join(lines)
    return lyrics

# Word Replacer

In [10]:
# MASKING

def enum_print_lines(lines):
    for i in range(len(lines)):
        print(i, ": ", lines[i])

def enum_print_fills(fills):
    for i in range(len(fills)):
        print(i, ": ", fills[i]['token_str'])

def in_line_i_change_word_j(i,j,lyrics):

    print("\n") 
    lines = lyrics.split("\n")
    enum_print_lines(lines)
    print("\n")

    # Masking the j th word of line i
    line = lines[i]
    words = line.split(" ")
    words[j] = "<mask>"
    line = ' '.join(words)
    mask_filler(line)
    print("\n")

    # Making suggestions
    fills = mask_filler(line)
    print(line)
    enum_print_fills(fills)
    print("\n")
    print(len(fills), ": ", "'I dont want to change'") 
    print(len(fills) + 1, ": ", "'I want to write the word(s) myself'") 
    print(len(fills) + 2, ": ", "'I want to erase the word'") 
    print("\n")
    choice = int(input("Give the number of the word of your choice: "))
    print("\n")

    # Restructuring the lyrics
    if choice < len(fills):
        words[j] = fills[choice]['token_str']
        words[j] = words[j].strip()
        line = ' '.join(words)
        lines[i] = line
    elif (choice == len(fills)):
        pass
    elif (choice == len(fills) + 1):
        words[j] = input("Write the your word(s) of your choice: ")
        words[j] = words[j].strip()
        line = ' '.join(words)
        lines[i] = line
    elif (choice == len(fills) + 2):
        words.remove(words[j])
        line = ' '.join(words)
        line = line.strip()
        lines[i] = line
    enum_print_lines(lines)
    print("\n")

    lyrics = '\n'.join(lines)
    return lyrics 
  
def change_words(lyrics):

    print("\n") 
    lines = lyrics.split("\n")
    enum_print_lines(lines)
    print("\n")

    #change = int(input("Do you want to change any words:\n0: no\n1: yes\nAnswer (with number): "))

    #if change == 0:
    #    return lyrics

    coord = input("Give the number of the line and number of the word from left you want to change\n\
(e.g.: '1,2' would be 2nd word of line number 1): ").split(",")
    coord = [int(i) for i in coord]

    lyrics = in_line_i_change_word_j(coord[0],coord[1]-1,lyrics)

    while "1" == input("Do you want to change another word?\n0: no\n1: yes\nAnswer: "):
        print("\n")
        coord = input("Give the number of the line and number of the word from left you want to change\n\
(e.g.: '1,2' would be 2nd word of line number 1): ").split(",")
        coord = [int(i) for i in coord] 
        lyrics = in_line_i_change_word_j(coord[0],coord[1]-1,lyrics)

    save_lyrics(lyrics)

    return lyrics

# Rhyming with "pronunciation"

In [11]:
# RHYMING

def enum_print_lines(lines):
    for i in range(len(lines)):
        print(i, ": ", lines[i])

def enum_print_rhymes(rhymes):
    for i in range(10):
        print(i, ": ", rhymes[i])

def rhyme_line_i_to_line_j_pro(j,i,lyrics):

    #j is base
    # i is to rhyme

    print("\n") 
    lines = lyrics.split("\n")
    enum_print_lines(lines)
    print("\n")

    # Masking the last word of line i
    line_i = lines[i]
    words_i = line_i.split(" ")
    words_i[-1] = "<mask>"
    line_i = ' '.join(words_i)

    line_j = lines[j]
    words_j = line_j.split(" ")
    print("\n")


    # Making suggestions
    rhymes = pronouncing.rhymes(words_j[-1])[:10]
    print("Base:     ",line_j)
    print("To rhyme: ",line_i)
    enum_print_rhymes(rhymes)
    print("\n")
    print(len(rhymes), ": ", "'I dont want to change'") 
    print(len(rhymes) + 1, ": ", "'I want to write the rhyme myself'") 
    print(len(rhymes) + 2, ": ", "'I want to erase the word'") 
    print("\n")
    choice = int(input("Give the number of the rhyming word of your choice: "))
    print("\n")

    # Restructuring the lyrics
    if choice < len(rhymes):
        words_i[-1] = rhymes[choice]
        line = ' '.join(words_i)
        lines[i] = line
    elif (choice == len(rhymes)):
        pass
    elif (choice == len(rhymes) + 1):
        words_i[-1] = input("Write the rhyming word(s) of your choice: ")
        words_i[-1] = words_i[-1].strip()
        line = ' '.join(words_i)
        lines[i] = line.strip()
    elif (choice == len(rhymes) + 2):
        words_i.remove(words_i[-1])
        line = ' '.join(words_i)
        line = line.strip()
        lines[i] = line
    enum_print_lines(lines)
    print("\n")

    lyrics = '\n'.join(lines)
    return lyrics 
  
def rhyme_words_pro(lyrics):

    print("\n") 
    lines = lyrics.split("\n")
    enum_print_lines(lines)
    print("\n")

    #change = int(input("Do you want to change any words to rhyme:\n0: no\n1: yes\nAnswer (with number): "))

    #if change == 0:
    #    return lyrics

    coord = input("Give the number of the line to base your rhyme on and line you want to change the\n\
last word (e.g.: '1,2' would rhyme line number 2 to line number 1): ").split(",")
    print("Taking 5 sec...")
    coord = [int(i) for i in coord]

    lyrics = rhyme_line_i_to_line_j_pro(coord[0],coord[1],lyrics)

    while "1" == input("Do you want to change any words to rhyme:\n0: no\n1: yes\nAnswer (with number): "):
        print("\n")
        coord = input("Give the number of the line to base your rhyme on and line you want to change the\n\
last word (e.g.: '1,2' would rhyme line number 2 to line number 1): ").split(",")
        print("Taking 5 sec...")
        coord = [int(i) for i in coord] 
        lyrics = rhyme_line_i_to_line_j_pro(coord[0],coord[1],lyrics)

    lyrics = save_lyrics(lyrics)

    return lyrics

# Rhyming with "RhymeZone.com"

In [21]:
base_url = "https://www.rhymezone.com/r/rhyme.cgi?Word={}&typeofrhyme=adv&loc=advlink"

def find_rhymes(word):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(options=chrome_options)
    #driver = webdriver.Chrome("chromedriver", options=chrome_options)

    formatted_url = base_url.format(quote_plus(word))

    response = driver.get(formatted_url)
    sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.select_one("table#rzadv_table")# >tbody>tr")

    rhymes = table.select("tr")[1:]
    driver.close()
    #print(soup.prettify())
    #print(rhymes)

    return list(map(table_element_to_rhyme_object, rhymes))
    #return [table_element_to_rhyme_object(rhyme) for rhyme in rhymes]


def table_element_to_rhyme_object(table_element):
    table_cells = table_element.select("td")

    word = table_cells[0].text
    rating = int(table_cells[1].text)
    categories = table_cells[4].text

    return {
      "word": word,
      "rating": rating,
      "categories": categories
    }

def rhyming_w_rhymezone(to_rhyme):
    #to_rhyme = input("Give a word to rhyme with: ")
    rhymes = []
    for word in find_rhymes(to_rhyme)[:10]:
        rhymes.append(word['word'])
        #print(word['word'], word['categories'])
    return rhymes

#if __name__ == "__main__":
#    to_rhyme = input("Give a word to rhyme with: ")
#    for word in find_rhymes(to_rhyme)[:10]:
#      print(word['word'], word['categories'])

4# RHYMING

def enum_print_lines(lines):
    for i in range(len(lines)):
        print(i, ": ", lines[i])

def enum_print_rhymes(rhymes):
    for i in range(len(rhymes)):
        print(i, ": ", rhymes[i])

def rhyme_line_i_to_line_j_rz(j,i,lyrics):

    #j is base
    # i is to rhyme

    print("\n") 
    lines = lyrics.split("\n")
    enum_print_lines(lines)
    print("\n")

    # Masking the last word of line i
    line_i = lines[i]
    words_i = line_i.split(" ")
    words_i[-1] = "<mask>"
    line_i = ' '.join(words_i)

    line_j = lines[j]
    words_j = line_j.split(" ")
    print("\n")


    # Making suggestions
    rhymes = rhyming_w_rhymezone(words_j[-1]) #pronouncing.rhymes(words_j[-1])[:10]
    print("Base:     ",line_j)
    print("To rhyme: ",line_i)
    enum_print_rhymes(rhymes)
    print("\n")
    print(len(rhymes), ": ", "'I dont want to change'") 
    print(len(rhymes) + 1, ": ", "'I want to write the rhyme myself'") 
    print(len(rhymes) + 2, ": ", "'I want to erase the word'") 
    print("\n")
    choice = int(input("Give the number of the rhyming word of your choice: "))
    print("\n")

    # Restructuring the lyrics
    if choice < len(rhymes):
        words_i[-1] = rhymes[choice]
        line = ' '.join(words_i)
        lines[i] = line
    elif (choice == len(rhymes)):
        pass
    elif (choice == len(rhymes) + 1):
        words_i[-1] = input("Write the rhyming word(s) of your choice: ")
        words_i[-1] = words_i[-1].strip()
        line = ' '.join(words_i)
        lines[i] = line.strip()
    elif (choice == len(rhymes) + 2):
        words_i.remove(words_i[-1])
        line = ' '.join(words_i)
        line = line.strip()
        lines[i] = line
    enum_print_lines(lines)
    print("\n")

    lyrics = '\n'.join(lines)
    return lyrics 
  
def rhyme_words_zone(lyrics):

    print("\n") 
    lines = lyrics.split("\n")
    enum_print_lines(lines)
    print("\n")

    #change = int(input("Do you want to change any words to rhyme:\n0: no\n1: yes\nAnswer (with number): "))

    #if change == 0:
    #    return lyrics

    coord = input("Give the number of the line to base your rhyme on and line you want to change the\n\
last word (e.g.: '1,2' would rhyme line number 2 to line number 1): ").split(",")
    print("Taking 5 sec...")
    coord = [int(i) for i in coord]

    lyrics = rhyme_line_i_to_line_j_rz(coord[0],coord[1],lyrics)

    while "1" == input("Do you want to change any other words to rhyme:\n0: no\n1: yes\nAnswer (with number): "):
        print("\n")
        coord = input("Give the number of the line to base your rhyme on and line you want to change the\n\
last word (e.g.: '1,2' would rhyme line number 2 to line number 1): ").split(",")
        print("Taking 5 sec...")
        coord = [int(i) for i in coord] 
        lyrics = rhyme_line_i_to_line_j_rz(coord[0],coord[1],lyrics)

    lyrics = save_lyrics(lyrics)

    return lyrics

# Lets write a song


In [22]:
def ask_menu_choice():
    print("\n")
    menu_choice = input("What do you want to do? \
    \n0: Swap lines\
    \n1: Replace words\
    \n2: Rhyme with 'pronuncation' (similar sounding syllables)\
    \n3: Rhyme with 'RhymeZone.com' (actual words)\
    \n4: I am done with my song!\
    \nAnswer (with number): ")
    print("\n")

    return menu_choice

def lets_write_a_song():

    start_choice = input("0: I have lyrics I am already working on..\n1: I want to start from scratch!\nAnswer (with numbers): ")
    print("\n")

    if start_choice == "0":
        lyrics = input("Copy paste your lyrics here (with 2 space between each line and 1 space between each word): ")
        lines = lyrics.split("  ")
        lyrics = "\n".join(lines)
        enum_print_lyrics(lyrics)
        lyrics = generate_lyrics_demo(lyrics)
    elif start_choice == "1":
        lyrics = ""
        lyrics = generate_lyrics_demo(lyrics)
        print("\n")

    menu_choice = ask_menu_choice()

    while menu_choice != "4":
        if menu_choice == "0":
            lyrics = swap_lines(lyrics)
        elif menu_choice == "1":
            lyrics = change_words(lyrics)
        elif menu_choice == "2":
            lyrics = rhyme_words_pro(lyrics)
        elif menu_choice == "3":
            lyrics = rhyme_words_zone(lyrics)
            
        enum_print_lyrics(lyrics)
        lyrics = save_lyrics(lyrics)
        print("\n")
        print("Saved!")
        
        menu_choice = ask_menu_choice()
    print("\n")
    print("Congratulations I hope you like your lyrics!")
    save_lyrics(lyrics)
    print("\n")
    print("Saved for the last time! ;)")

    return lyrics

# DEMO

For "0 : I have lyrics I am already working on.." you can copy paste the following output:

In [23]:
print("I don't know where it leads  I think I'll keep thinking of you  I'll keep thinking of you  You know I'm always the one")
# Triple click a word below to select easily

I don't know where it leads  I think I'll keep thinking of you  I'll keep thinking of you  You know I'm always the one



Example prompt (you can copy paste the following output):

In [24]:
print("To get out of the Matrix") # Triple click a word below to select easily
# 3: Metal goes well with this prompt  
# (BEWARE may produce controvertial lines, metal sometimes has strong language)

To get out of the Matrix


In [25]:
load_gen_lyrics() # comment this back in if you got an error but want continue with the last lyrics
# Triple click a word below to select easily

0 :  I don't know where it leads
1 :  I think I'll keep thinking of you
2 :  I'll keep thinking of you
3 :  You know I'm always the one




"I don't know where it leads  I think I'll keep thinking of you  I'll keep thinking of you  You know I'm always the one"

In [27]:
# IF YOU GET AN ERROR AND WANT TO KEEP GOING WITH THE SAME SONG
# RUN ABOVE CELL AND COPY PASTE THE OUTPUT (AFTER CHOOSING 0)
# ERASING the " AT THE START AND THE END
lyrics = ""
lyrics = lets_write_a_song()





Your lyrics:
0 :  flying over the clouds
1 :  Just like that river twisting through a dusty land
2 :  And when she shines she really shows you all she can - oh, you got to be mine
3 :  She really got to be mine
4 :  She said so
5 :  I think youre gonna be mine
6 :  She said so
7 :  I think youre gonna be
8 :  I think youre gonna be
9 :  Well tell ya
10 :  I think I'm gonna be mine
11 :  Well tell ya
12 :  flying over the clouds
13 :  Just a fun, quirky, quirky shot
14 :  Bad luck, bad luck, bad luck
15 :  So bad luck, bad luck,
16 :  Have you ever flown before
17 :  Have you flew before - yeah
18 :  Have you ever flown before
19 :  Do you think you fit this shoe




Setting up genre specific model...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertLMHeadModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertLMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertLMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertLMHeadModel were not initialized from the PyTorch model and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it fo

Model Path: /Users/tatlikazan-rv/Documents/thesis_documents/relevant-projects/Echo_Canyon/saved_model/country
Done!




Your lyrics:
0 :  flying over the clouds
1 :  Just like that river twisting through a dusty land
2 :  And when she shines she really shows you all she can - oh, you got to be mine
3 :  She really got to be mine
4 :  She said so
5 :  I think youre gonna be mine
6 :  She said so
7 :  I think youre gonna be
8 :  I think youre gonna be
9 :  Well tell ya
10 :  I think I'm gonna be mine
11 :  Well tell ya
12 :  flying over the clouds
13 :  Just a fun, quirky, quirky shot
14 :  Bad luck, bad luck, bad luck
15 :  So bad luck, bad luck,
16 :  Have you ever flown before
17 :  Have you flew before - yeah
18 :  Have you ever flown before
19 :  Do you think you fit this shoe




Your lyrics:
0 :  flying over the clouds
1 :  Just like that river twisting through a dusty land
2 :  And when she shines she really shows you all she can - oh, you got to be mine
3 :  She really got to be

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Base:      Just like that river twisting through a dusty land
To rhyme:  And when she shines she really shows you all she can - oh, you got to be <mask>
0 :  hand
1 :  understand
2 :  demand
3 :  stand
4 :  command
5 :  band
6 :  Grand
7 :  planned
8 :  sand
9 :  expand


10 :  'I dont want to change'
11 :  'I want to write the rhyme myself'
12 :  'I want to erase the word'




0 :  flying over the clouds
1 :  Just like that river twisting through a dusty land
2 :  And when she shines she really shows you all she can - oh, you got to be hand
3 :  She really got to be mine
4 :  She said so
5 :  I think youre gonna be mine
6 :  She said so
7 :  I think youre gonna be
8 :  I think youre gonna be
9 :  Well tell ya
10 :  I think I'm gonna be mine
11 :  Well tell ya
12 :  flying over the clouds
13 :  Just a fun, quirky, quirky shot
14 :  Bad luck, bad luck, bad luck
15 :  So bad luck, bad luck,
16 :  Have you ever flown before
17 :  Have you flew before - yeah
18 :  Have you ever flown befor

In [ ]:
lyrics = load_gen_lyrics()

In [ ]:
lyrics

In [ ]:
lines = lyrics.split("  ")
lyrics = "\n".join(lines)

In [ ]:
lyrics